In [6]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
import pprint
import glob
import logging
import pickle
import sys
from pprint import pformat

from sacred import Experiment
from sacred.observers import MongoObserver

import pathpy
import numpy as np

from scipy.stats import chi2
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from matplotlib.pyplot import figure

In [3]:
root_dir = os.curdir
max_nest = 10  # arbitrary, 3 would probably suffice
nest = 0
while "src" not in os.listdir(root_dir) and nest < max_nest:
    # Look up the directory structure for a src directory
    root_dir = os.path.join(os.pardir, root_dir)
    nest += 1

# If you don't find the src directory, the root directory is this directory
root_dir = os.path.abspath(root_dir) if nest < max_nest else os.path.abspath(
    os.curdir)

# Get the source directory and append path to access
# python packages/scripts within directory
if "src" in os.listdir(root_dir):
    src_dir = os.path.join(root_dir)
    sys.path.append(src_dir)

In [5]:
from src.preprocess_experiment import get_dataset
from src.data_processing import generate_temporal_network, get_runs

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'

In [5]:
net = generate_temporal_network("../data/raw/CVE-2017-7529/aggressive_brahmagupta_2425.txt")
print(net)

2020-09-04 10:56:50 [Severity.INFO]	Building index data structures ...
2020-09-04 10:56:50 [Severity.INFO]	Sorting time stamps ...
2020-09-04 10:56:50 [Severity.INFO]	finished.
Nodes:			19
Time-stamped links:	1527
Links/Nodes:		80.36842105263158
Observation period:	[0, 49489242]
Observation length:	 49489242 
Time stamps:		 1527 
Avg. inter-event dt:	 32430.695937090433
Min/Max inter-event dt:	 1/5548078


In [24]:
paths = pickle.load(open("../data/interim/CVE-2017-7529/temp_paths_10.p", "rb"))

In [51]:
print(paths)

print(paths.paths[4])

Total path count: 		1068194.0 
[Unique / Sub paths / Total]: 	[320.0 / 3740070.0 / 4808264.0]
Nodes:				19 
Edges:				69
Max. path length:		20
Avg path length:		1.418005530830542 
Paths of length k = 0		0.0 [ 0.0 / 2582899.0 / 2582899.0 ]
Paths of length k = 1		817870.0 [ 54.0 / 696835.0 / 1514705.0 ]
Paths of length k = 2		91434.0 [ 46.0 / 355077.0 / 446511.0 ]
Paths of length k = 3		132989.0 [ 42.0 / 63198.0 / 196187.0 ]
Paths of length k = 4		20848.0 [ 36.0 / 16449.0 / 37297.0 ]
Paths of length k = 5		2429.0 [ 32.0 / 8967.0 / 11396.0 ]
Paths of length k = 6		1620.0 [ 23.0 / 4723.0 / 6343.0 ]
Paths of length k = 7		61.0 [ 25.0 / 3658.0 / 3719.0 ]
Paths of length k = 8		83.0 [ 20.0 / 2632.0 / 2715.0 ]
Paths of length k = 9		709.0 [ 14.0 / 1063.0 / 1772.0 ]
Paths of length k = 10		15.0 [ 9.0 / 897.0 / 912.0 ]
Paths of length k = 11		62.0 [ 14.0 / 699.0 / 761.0 ]
Paths of length k = 12		2.0 [ 1.0 / 623.0 / 625.0 ]
Paths of length k = 13		0.0 [ 0.0 / 551.0 / 551.0 ]
Paths of length k = 1

In [46]:
hon_1 = pathpy.HigherOrderNetwork(paths, k=1)
#print(hon_1.transition_matrix())

In [80]:
mog = pathpy.MultiOrderModel(paths, max_order=3)
print(mog.paths.paths.keys())
#order = mog.estimate_order()
print(order)

2020-09-04 11:17:57 [Severity.INFO]	Generating 0-th order layer ...
2020-09-04 11:17:57 [Severity.INFO]	Generating 1-th order layer ...
2020-09-04 11:17:57 [Severity.INFO]	Generating 2-th order layer ...
2020-09-04 11:17:57 [Severity.INFO]	Generating 3-th order layer ...
2020-09-04 11:17:57 [Severity.INFO]	finished.
dict_keys([1, 2, 3, 4, 9, 6, 0, 5, 7, 8, 11, 10, 20, 12, 13, 14, 15, 16, 17, 18, 19])
3


In [83]:
mypaths = pathpy.Paths()
mypaths.add_path(["open","stat"])
mypaths.add_path(['epoll_wait', 'epoll_wait', 'recvmsg', 'close', 'recvmsg'])
mypaths.add_path(["stat","open"])
mypaths.add_path(["stat","open"])
mypaths.add_path(["stat","open"])
mypaths.add_path(["stat","open"])
print(mypaths)
#mypaths.add_path(["futex","epoll_wait"])
print(hon_1.likelihood(mypaths, log=True))
print(mog.likelihood(mypaths, log=True))
hon_1.model_size()

Total path count: 		6.0 
[Unique / Sub paths / Total]: 	[3.0 / 24.0 / 30.0]
Nodes:				5 
Edges:				6
Max. path length:		4
Avg path length:		1.5 
Paths of length k = 0		0.0 [ 0.0 / 15.0 / 15.0 ]
Paths of length k = 1		5.0 [ 2.0 / 4.0 / 9.0 ]
Paths of length k = 2		0.0 [ 0.0 / 3.0 / 3.0 ]
Paths of length k = 3		0.0 [ 0.0 / 2.0 / 2.0 ]
Paths of length k = 4		1.0 [ 1.0 / 0.0 / 1.0 ]

-inf
-inf


/home/jp/miniconda3/envs/hids/lib/python3.8/site-packages/pathpy/classes/higher_order_network.py:449: RuntimeWarning: divide by zero encountered in log
  path_L += _np.log(T[node_map[n], node_map[prev]])
/home/jp/miniconda3/envs/hids/lib/python3.8/site-packages/pathpy/classes/multi_order_model.py:547: RuntimeWarning: divide by zero encountered in log
  likelihood += np.log(trans_mat) * freq


69

In [96]:
print(mypaths)
print(mog.likelihood(mypaths, log=False))
mog.model_size(0)
mog.transition_matrices[0]

Total path count: 		1.0 
[Unique / Sub paths / Total]: 	[1.0 / 0.0 / 1.0]
Nodes:				1 
Edges:				0
Max. path length:		0
Avg path length:		0.0 
Paths of length k = 0		1.0 [ 1.0 / 0.0 / 1.0 ]

0.0006988271705552563


<20x20 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [12]:
style = { 
    'label_offset': [0,-1], 
    'label_color' : 'black', 
    'width': 800, 
    'height': 800 
}
pathpy.visualisation.plot(hon_1, **style)


for e in hon_1.edges:
    print(e, hon_1.edges[e]['weight'])
    

('pread', 'writev') [56228. 50747.]
('writev', 'sendfile') [   759. 106217.]
('sendfile', 'write') [  3814. 103161.]
('epoll_ctl', 'open') [  3808. 103165.]
('open', 'fstat') [50146. 57980.]
('fstat', 'pread') [56345. 50628.]
('recvfrom', 'epoll_ctl') [52289. 54685.]
('close', 'epoll_wait') [59205. 47466.]
('accept', 'epoll_ctl') [22986. 83994.]
('epoll_wait', 'accept') [  3590. 103358.]
('epoll_ctl', 'epoll_wait') [92099. 14345.]
('close', 'close') [97651.  9721.]
('epoll_wait', 'recvfrom') [92911. 13512.]
('write', 'close') [96110. 10865.]
('epoll_wait', 'epoll_wait') [  57. 3508.]
('epoll_ctl', 'recvfrom') [197. 340.]
('epoll_wait', 'futex') [0. 3.]
('getdents', 'brk') [844. 308.]
('rt_sigsuspend', 'wait4') [  0. 330.]
('epoll_wait', 'open') [  5. 379.]
('epoll_wait', 'recvmsg') [218. 138.]
('recvmsg', 'close') [489.  84.]
('close', 'recvmsg') [537.  65.]
('close', 'rt_sigsuspend') [10.  4.]
('getdents', 'stat') [499. 653.]
('stat', 'open') [338. 430.]
('fstat', 'getdents') [448. 31

In [27]:
hon_2 = pathpy.HigherOrderNetwork(paths, k=2, null_model=False)
pathpy.visualisation.plot(hon_2, **style)

for e in hon_2.edges:
    print(e, hon_2.edges[e]['weight'])

test_paths = pathpy.HigherOrderNetwork.generate_possible_paths(hon_2, 2)
print(len(test_paths))

('write,close', 'close,close') [58218. 37885.]
('epoll_ctl,epoll_wait', 'epoll_wait,recvfrom') [50313. 40754.]
('fstat,pread', 'pread,writev') [49264.  6963.]
('close,close', 'close,epoll_wait') [58034.  1157.]
('epoll_ctl,epoll_wait', 'epoll_wait,accept') [366. 149.]
('open,fstat', 'fstat,pread') [49268.   114.]
('epoll_wait,recvfrom', 'recvfrom,epoll_ctl') [50526.  1589.]
('accept,epoll_ctl', 'epoll_ctl,epoll_wait') [22276.   294.]
('epoll_wait,accept', 'accept,epoll_ctl') [3383.   58.]
('sendfile,write', 'write,close') [3076.    7.]
('recvfrom,epoll_ctl', 'epoll_ctl,open') [3787.   13.]
('epoll_ctl,open', 'open,fstat') [820.   0.]
('writev,sendfile', 'sendfile,write') [ 28. 731.]
('getdents,stat', 'stat,open') [300.  37.]
('open,fstat', 'fstat,getdents') [150. 298.]
('getdents,stat', 'stat,getdents') [137.  25.]
('close,close', 'close,sendmsg') [  0. 326.]
('wait4,wait4', 'wait4,close') [  0. 325.]
('epoll_wait,recvmsg', 'recvmsg,close') [76. 24.]
('recvmsg,close', 'close,recvmsg') 

In [ ]:
style = {    
  'ts_per_frame': 100, 
  'ms_per_frame': 10,
  'look_ahead': 100, 
  'look_behind': 100, 
  'node_size': 15, 
  'inactive_edge_width': 2,
  'active_edge_width': 4, 
  'label_color' : '#00000',
  'label_size' : '24px',
  'label_offset': [0,10],
  'width': 800,
  'height': 800,
  }
#pathpy.visualisation.plot(temp_net_true, **style)